In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from concurrent.futures import ThreadPoolExecutor
currdir = os.getcwd()
parent = os.path.dirname(currdir)
gparent = os.path.dirname(parent)
parent

'/home/poojaparameswaran/Documents/SoderlingLab/SpatialProteomics'

In [4]:
KOGENE = 'LRRK2'
modules_file = f"{parent}/example_data/tables/{KOGENE}-TMT-Module-Results.xlsx"
module_df = pd.read_excel(modules_file, engine='openpyxl')
module_df

,UniProt,Entrez,Symbol,Membership
0,A2A5R2,99371,Arfgef2,0
1,E9PY46,106633,Ift140,0
2,O54829,24012,Rgs7,0
3,O70325,625249,Gpx4,0
4,P43274,50709,H1f4,0
...,...,...,...,...
1952,P28658,54138,Atxn10,45
1953,Q9CVB6,76709,Arpc2,45
1954,Q9WVG6,59035,Carm1,45
1955,Q9Z1Z0,56041,Uso1,45


In [5]:
genes = list(module_df['Entrez'])

# url = 'https://rest.kegg.jp/conv/mmu/ncbi-geneid:'
url = 'https://rest.kegg.jp/link/ko/'


In [6]:
def get_kegg_id(entrezIDs):
    KOs = {}
    formatted_ids = "+".join(["mmu:" + str(id) for id in entrezIDs])
    response = requests.get(url +formatted_ids)
    # Parse the response to extract the KEGG ID
    if response.ok:
        lines = response.text.split('\n')
        KOs = {f"mmu:{id}": next((line.split('\t')[1].strip() for line in lines if f"mmu:{id}" in line), "FAIL") for id in entrezIDs}

    return KOs

In [7]:

batch_size = 100

### ALREADY HAVE KEGGIDS... -> entrez.
all_KOs = {}
for i in tqdm(range(0, len(genes), batch_size)):    # Wrap executor.map with tqdm for the progress bar
    batch = genes[i:i+batch_size]
    all_KOs.update(get_kegg_id(batch))

100%|██████████| 20/20 [00:26<00:00,  1.31s/it]


In [8]:
module_df['KEGG_Ortholog'] = module_df['Entrez'].map(lambda x: all_KOs.get(f"mmu:{x}", "No KEGG ID"))
print(len(all_KOs.keys()))
module_df['KEGG_Ortholog'] = module_df['KEGG_Ortholog'].apply(lambda x: x.split(":")[1] if "ko:" in x else x)
module_df

1957


,UniProt,Entrez,Symbol,Membership,KEGG_Ortholog
0,A2A5R2,99371,Arfgef2,0,K18442
1,E9PY46,106633,Ift140,0,K19672
2,O54829,24012,Rgs7,0,K16449
3,O70325,625249,Gpx4,0,K05361
4,P43274,50709,H1f4,0,K11275
...,...,...,...,...,...
1952,P28658,54138,Atxn10,45,K19323
1953,Q9CVB6,76709,Arpc2,45,K05758
1954,Q9WVG6,59035,Carm1,45,K05931
1955,Q9Z1Z0,56041,Uso1,45,FAIL


In [10]:
module_df.to_csv(f"{parent}/example_data/tables/{KOGENE}_moduleResults_KEGGortholog.csv") ## this is going in example_data/ change it to your tables/ dir. 